# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [31]:
import requests as re
from IPython.display import JSON

import tfl_env as env

## Demo

In [2]:
app_id = env.PRIMARY_KEY
app_key = env.SECONDARY_KEY
url_append = f'?app_id={app_id}&app_key={app_key}' 

In [3]:
print(url_append)

?app_id=e5d1123c6c2e491c98b70b1f3e943b87&app_key=c2bfa03caef04faca902f1a58a966c69


In [4]:
def get_tfl(endpoint, **params):
    params["app_id"] = app_id
    params["app_key"] = app_key
    return re.get(
        f"https://api.tfl.gov.uk/{endpoint}",
        params
    )

In [5]:
res = get_tfl("AirQuality")

In [6]:
res.status_code

200

In [7]:
air = res.json()

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [8]:
def find_with_value(ls, key, value):
    """
    In list of dictionaries ``ls``, find the first dictionary
    that maps ``key`` to ``value``
    """
    for d in ls:
        if key in d and d[key] == value:
            return d

In [9]:
forecast = find_with_value(
    air['currentForecast'], 'forecastType', 'Future'
)
pollutant_types = {
    'Nitrogen Dioxide': 'nO2Band',
    'Ozone': 'o3Band',
    'PM10 Particulate': 'pM10Band',
    'PM2.5 Particulate': 'pM25Band',
    'Sulphur Dioxide': 'sO2Band',
}
forecast_string = "\n".join(
    f"{name}: {forecast[key]}" for name, key in pollutant_types.items()
)
print(forecast_string)

Nitrogen Dioxide: Low
Ozone: Low
PM10 Particulate: Low
PM2.5 Particulate: Low
Sulphur Dioxide: Low


## Task
What are the different modes of transport which are operated by Transport for London? How many modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [10]:
mode_res = get_tfl("Line/Meta/Modes")

In [11]:
mode_json = mode_res.json()

In [12]:
modes = [mode['modeName'] for mode in mode_json if mode['isTflService']]
print(f"[{', '.join(modes)}]")
print(f"There are {len(modes)} different modes of transport")

[bus, cable-car, cycle-hire, dlr, overground, replacement-bus, river-bus, river-tour, tflrail, tram, tube]
There are 11 different modes of transport


## Task
How many BikePoints in London are operated by Transport for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [13]:
bike_res = get_tfl("BikePoint")

In [14]:
bike_json = bike_res.json()
print(f"There are {len(bike_json)} BikePoints in London")

There are 792 BikePoints in London


In [15]:
total_docks = sum(
    int(
        find_with_value(
            bike_point['additionalProperties'], 'key', 'NbDocks'
        )['value']
    )
    for bike_point in bike_json
)
print(f"There are {total_docks} total docks at all BikePoints")

There are 21056 total docks at all BikePoints


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [16]:
tube_line_res = get_tfl("Line/Mode/tube")

In [17]:
bus_line_res = get_tfl("Line/Mode/bus")

In [18]:
tube_json = tube_line_res.json()
bus_json = bus_line_res.json()
print(f"There are {len(tube_json)} tube lines and {len(bus_json)} bus lines")

There are 11 tube lines and 679 bus lines


In [19]:
for line in tube_json:
    print(line['name'])

Bakerloo
Central
Circle
District
Hammersmith & City
Jubilee
Metropolitan
Northern
Piccadilly
Victoria
Waterloo & City


## Task
How many stations are on the `Victoria` line?

In [20]:
victoria_id = find_with_value(tube_json, 'name', 'Victoria')['id']
victoria_res = get_tfl(f"Line/{victoria_id}/StopPoints")

In [21]:
victoria_json = victoria_res.json()
print(f"There are {len(victoria_json)} stations on the Victoria line")

There are 16 stations on the Victoria line


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube. Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [35]:
tube_journey_res_ambiguous = get_tfl(
    "Journey/JourneyResults/HeathrowAirport/to/TowerBridge",
    mode="tube",
)

In [32]:
JSON(tube_journey_res_ambiguous.json())

<IPython.core.display.JSON object>

In [33]:
# "Heathrow Airport" is ambiguous. Use the lat/long for Terminal 2:
heathrow_airport = "51.46962511264,-0.44968473098"
# "Tower Bridge" is ambiguous. Use the lat/long:
tower_bridge = "51.50599630145,-0.07502752221"

In [36]:
tube_journey_res_partly_ambiguous = get_tfl(
    "Journey/JourneyResults/{heathrow_airport}/to/TowerBridge",
    mode="tube",
)

In [37]:
JSON(tube_journey_res_partly_ambiguous.json())

<IPython.core.display.JSON object>

In [39]:
postcode_to_text = get_tfl("Journey/JourneyResults/NW10 1ND/to/TowerBridge")

In [40]:
JSON(postcode_to_text.json())

<IPython.core.display.JSON object>

In [24]:
tube_journey_res = get_tfl(
    f"Journey/JourneyResults/{heathrow_airport}/to/{tower_bridge}",
    mode="tube",
    time="1500",
    timeIs="departing",
)

In [38]:
tube_journey_json = tube_journey_res.json()
JSON(tube_journey_json)

<IPython.core.display.JSON object>

In [41]:
tube_journey_time = (
    tube_journey_json['journeys'][0]['duration']
)
tube_journey_time

80

In [26]:
bus_journey_res = get_tfl(
    f"Journey/JourneyResults/{heathrow_airport}/to/{tower_bridge}",
    mode="bus",
    time="1500",
    timeIs="departing"
)

In [43]:
bus_journey_json = bus_journey_res.json()
bus_journey_time = (
    bus_journey_json['journeys'][0]['duration']
)
bus_journey_time

211

In [44]:
print("Planned Duration:")
print(f"Bus: {bus_journey_time} minutes")
print(f"Tube: {tube_journey_time} minutes")

Planned Duration:
Bus: 211 minutes
Tube: 80 minutes
